# Hack Napoli V2

## Definizione delle classi

In [1]:
class SpaceShip:
    def __init__(self, planet, fuel, container_num):
        self.planet = planet
        self.fuel = fuel
        self.container_num = container_num
    def _repr_(self):
        return "<Test Planet:%s, Fuel:%s, ContainerNum:%s>" % (self.planet, 
                                                               self.fuel, 
                                                               self.container_num)
    def _str_(self):
        return "From str method of Test: Planet is %s, Fuel is %s, ContainerNum is %s" % (self.planet, 
                                                                                          self.fuel, 
                                                                                          self.container_num)

class Container:
    def __init__(self, id, value, capacity):
        self.id = id
        self.value = value
        self.capacity = capacity
    def __repr__(self):
        return "<Test Id:%s Value:%s Capacity:%s>" % (self.id, 
                                                      self.value, 
                                                      self.capacity)
    def __str__(self):
        return "From str method of Test: Id is %s, Value is %s, Capacity is %s" % (self.id, 
                                                                                   self.value, 
                                                                                   self.capacity)

    
class Planet:
    def __init__(self, id, liquid, visited):
        self.id = id
        self.liquid = liquid
        self.visited = visited
    def __repr__(self):
        return "<Test Id:%s Liquid:%s isVisited?:%s>" % (self.id, 
                                                         self.liquid, 
                                                         self.visited)
    def __str__(self):
        return "From str method of Test: Id is %s, Liquid is %s, isVisited?:%s" % (self.id, 
                                                                                   self.liquid, 
                                                                                   self.visited)

class Link:
    def __init__(self, departure, destination, fuel):
        self.departure = departure
        self.destination = destination
        self.fuel = fuel
    def __repr__(self):
        return "<Test Departure:%s Destination:%s FuelNecessary:%s>" % (self.departure, 
                                                                        self.destination, 
                                                                        self.fuel)
    def __str__(self):
        return "From str method of Test: Departure is %s, Destination is %s, FuelNecessary is %s" % (self.departure, 
                                                                                                     self.destination, 
                                                                                                     self.fuel)
    

## Lettura delle costanti

In [2]:
import numpy as np

File = open("input","r")
Lines = File.readlines()
TotalLinesNum = len(Lines)

#--------------------INIZIALIZZAZIONE DELLE COSTANTI-----------------------

ConstLine = Lines[0].split(" ")
PlanetNum = int(ConstLine[0])
FuelMax = int(ConstLine[1])
ContainerNum = int(ConstLine[2])
LiquidNum = ContainerNum

#----------------------DEFINIZIONE DEI RANGE-------------------------------

PlanetsId = range(PlanetNum)
ContainersId = range(ContainerNum)
LiquidsId = range(LiquidNum)

#------------------------INIZIALIZZAZIONE DEI VETTORI----------------------

Planets = np.empty(PlanetNum, dtype = Planet)
Containers = np.empty(ContainerNum, dtype = Container)
spaceship = SpaceShip(int(0), FuelMax, ContainerNum)

## Popolamento dei vettori

In [3]:
#--------------------POPOLO IL VETTORE DEI CONTAINER----------------------

ContainerLine = Lines[1].split(" ")
for i in ContainersId:
    Containers[i] = Container(i,0,int(ContainerLine[i]))

#-------------------POPOLO IL VETTORE DEI PIANETI---------------------------

PlanetLineIndex = 2
for i in PlanetsId:
    PlanetLine = Lines[PlanetLineIndex].split(" ")
    LiqQty = {0: int(PlanetLine[1]), 
              1: int(PlanetLine[2]), 
              2: int(PlanetLine[3]), 
              3: int(PlanetLine[4]), 
              4: int(PlanetLine[5])}
    Planets[i] = Planet(int(PlanetLine[0]),LiqQty, 0)
    PlanetLineIndex = PlanetLineIndex + 1

#-------------POPOLO IL VETTORE DEI COLLEGAMENTI TRA I PIANETI-------------------

LinksLineNum = int(TotalLinesNum - PlanetLineIndex)
Links = []

NumOfLine = PlanetLineIndex

for k in range(LinksLineNum):
    LinkLine = Lines[NumOfLine].split(" ")
    Links.append(Link(int(LinkLine[0]), int(LinkLine[1]), int(LinkLine[2])))
    Links.append(Link(int(LinkLine[1]), int(LinkLine[0]), int(LinkLine[2])))
    NumOfLine = NumOfLine + 1

File.close()
print(len(Links))

160448


## Algoritmo risolutivo del problema

In [24]:
import math
from itertools import combinations
from datetime import datetime

SinglePlanet = []
PlanetRank = {}
ValidLinks = []

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

while (spaceship.fuel > 0):

    for link in Links:
        if (spaceship.planet == link.departure):
            ValidLinks.append(link)
            cost = sum(Planets[link.destination].liquid.values())
            rank = cost - link.fuel

            if (Planets[link.destination].visited == 0):
                PlanetRank[link.destination] = rank

            if (Planets[link.destination].visited == 1):
                # ---penalizzazione del pianeta con un rank molto basso----
                PlanetRank[link.destination] = -4000

    max_key = max(PlanetRank, key=PlanetRank.get)

    spaceship.planet = max_key
    for ValidLink in ValidLinks:
        if (ValidLink.destination == spaceship.planet):
            spaceship.fuel = spaceship.fuel - ValidLink.fuel
            break

    if (max_key not in SinglePlanet):
        SinglePlanet.append(max_key)
        for i in ContainersId:
            if Containers[i].value + Planets[spaceship.planet].liquid[i] <= Containers[i].capacity:
                Containers[i].value = Containers[i].value + Planets[spaceship.planet].liquid[i]

    print("Benzina rimanente: " + str(spaceship.fuel) + ". Sono sul pianeta: " + str(spaceship.planet))
    PlanetRank.clear()
    Planets[spaceship.planet].visited = 1


Score = (0.4 * len(SinglePlanet)) + (0.6 * sum(Containers[i].value for i in ContainersId))
print("Liquidi totali raccolti: %s" % sum(Containers[i].value for i in ContainersId))
print("Numero di nodi singoli visitati: " + str(len(SinglePlanet)))
print("Punteggio ottenuto: " + str(Score))
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
#-----------------------------------------------------------------------------------------------

2021-07-02 18:05:08
Benzina rimanente: 49997. Sono sul pianeta: 14908
Benzina rimanente: 49994. Sono sul pianeta: 0
Benzina rimanente: 49992. Sono sul pianeta: 546
Benzina rimanente: 49991. Sono sul pianeta: 8106
Benzina rimanente: 49990. Sono sul pianeta: 7711
Benzina rimanente: 49987. Sono sul pianeta: 14820
Benzina rimanente: 49986. Sono sul pianeta: 3088
Benzina rimanente: 49984. Sono sul pianeta: 9559
Benzina rimanente: 49978. Sono sul pianeta: 16044
Benzina rimanente: 49976. Sono sul pianeta: 12649
Benzina rimanente: 49967. Sono sul pianeta: 7824
Benzina rimanente: 49961. Sono sul pianeta: 952
Benzina rimanente: 49959. Sono sul pianeta: 1974
Benzina rimanente: 49958. Sono sul pianeta: 10902
Benzina rimanente: 49956. Sono sul pianeta: 18657
Benzina rimanente: 49954. Sono sul pianeta: 6443
Benzina rimanente: 49953. Sono sul pianeta: 12189
Benzina rimanente: 49949. Sono sul pianeta: 13877
Benzina rimanente: 49946. Sono sul pianeta: 6692
Benzina rimanente: 49945. Sono sul pianeta: 99

Benzina rimanente: 49487. Sono sul pianeta: 17289
Benzina rimanente: 49486. Sono sul pianeta: 5151
Benzina rimanente: 49484. Sono sul pianeta: 16864
Benzina rimanente: 49483. Sono sul pianeta: 9244
Benzina rimanente: 49482. Sono sul pianeta: 1593
Benzina rimanente: 49481. Sono sul pianeta: 4484
Benzina rimanente: 49471. Sono sul pianeta: 11810
Benzina rimanente: 49470. Sono sul pianeta: 11009
Benzina rimanente: 49465. Sono sul pianeta: 11434
Benzina rimanente: 49464. Sono sul pianeta: 8129
Benzina rimanente: 49459. Sono sul pianeta: 7950
Benzina rimanente: 49449. Sono sul pianeta: 3505
Benzina rimanente: 49448. Sono sul pianeta: 2189
Benzina rimanente: 49446. Sono sul pianeta: 1152
Benzina rimanente: 49445. Sono sul pianeta: 13447
Benzina rimanente: 49442. Sono sul pianeta: 9429
Benzina rimanente: 49441. Sono sul pianeta: 7020
Benzina rimanente: 49440. Sono sul pianeta: 13875
Benzina rimanente: 49439. Sono sul pianeta: 9852
Benzina rimanente: 49436. Sono sul pianeta: 6519
Benzina riman

Benzina rimanente: 48917. Sono sul pianeta: 1849
Benzina rimanente: 48907. Sono sul pianeta: 12720
Benzina rimanente: 48902. Sono sul pianeta: 3254
Benzina rimanente: 48901. Sono sul pianeta: 16201
Benzina rimanente: 48899. Sono sul pianeta: 17054
Benzina rimanente: 48898. Sono sul pianeta: 19425
Benzina rimanente: 48896. Sono sul pianeta: 3189
Benzina rimanente: 48893. Sono sul pianeta: 11702
Benzina rimanente: 48892. Sono sul pianeta: 3369
Benzina rimanente: 48891. Sono sul pianeta: 12439
Benzina rimanente: 48890. Sono sul pianeta: 16639
Benzina rimanente: 48888. Sono sul pianeta: 15548
Benzina rimanente: 48885. Sono sul pianeta: 13896
Benzina rimanente: 48881. Sono sul pianeta: 1908
Benzina rimanente: 48877. Sono sul pianeta: 1791
Benzina rimanente: 48873. Sono sul pianeta: 5956
Benzina rimanente: 48868. Sono sul pianeta: 3276
Benzina rimanente: 48867. Sono sul pianeta: 9756
Benzina rimanente: 48865. Sono sul pianeta: 4627
Benzina rimanente: 48862. Sono sul pianeta: 5199
Benzina rim

Benzina rimanente: 48328. Sono sul pianeta: 8289
Benzina rimanente: 48323. Sono sul pianeta: 9029
Benzina rimanente: 48322. Sono sul pianeta: 14358
Benzina rimanente: 48319. Sono sul pianeta: 16847
Benzina rimanente: 48316. Sono sul pianeta: 620
Benzina rimanente: 48315. Sono sul pianeta: 2318
Benzina rimanente: 48313. Sono sul pianeta: 1190
Benzina rimanente: 48311. Sono sul pianeta: 12800
Benzina rimanente: 48308. Sono sul pianeta: 18965
Benzina rimanente: 48307. Sono sul pianeta: 11357
Benzina rimanente: 48305. Sono sul pianeta: 15123
Benzina rimanente: 48302. Sono sul pianeta: 15888
Benzina rimanente: 48295. Sono sul pianeta: 11326
Benzina rimanente: 48290. Sono sul pianeta: 2299
Benzina rimanente: 48289. Sono sul pianeta: 3026
Benzina rimanente: 48288. Sono sul pianeta: 1441
Benzina rimanente: 48282. Sono sul pianeta: 16849
Benzina rimanente: 48278. Sono sul pianeta: 2140
Benzina rimanente: 48275. Sono sul pianeta: 6804
Benzina rimanente: 48270. Sono sul pianeta: 19950
Benzina rim

Benzina rimanente: 47683. Sono sul pianeta: 19641
Benzina rimanente: 47682. Sono sul pianeta: 11693
Benzina rimanente: 47677. Sono sul pianeta: 6231
Benzina rimanente: 47676. Sono sul pianeta: 10651
Benzina rimanente: 47675. Sono sul pianeta: 4653
Benzina rimanente: 47673. Sono sul pianeta: 5346
Benzina rimanente: 47670. Sono sul pianeta: 3467
Benzina rimanente: 47667. Sono sul pianeta: 1835
Benzina rimanente: 47666. Sono sul pianeta: 4154
Benzina rimanente: 47661. Sono sul pianeta: 19675
Benzina rimanente: 47653. Sono sul pianeta: 19494
Benzina rimanente: 47649. Sono sul pianeta: 19215
Benzina rimanente: 47647. Sono sul pianeta: 16436
Benzina rimanente: 47642. Sono sul pianeta: 19572
Benzina rimanente: 47640. Sono sul pianeta: 5879
Benzina rimanente: 47638. Sono sul pianeta: 8803
Benzina rimanente: 47636. Sono sul pianeta: 19307
Benzina rimanente: 47635. Sono sul pianeta: 3247
Benzina rimanente: 47634. Sono sul pianeta: 13261
Benzina rimanente: 47632. Sono sul pianeta: 19177
Benzina r

...

Benzina rimanente: 10166. Sono sul pianeta: 10112
Benzina rimanente: 10153. Sono sul pianeta: 426
Benzina rimanente: 10142. Sono sul pianeta: 19027
Benzina rimanente: 10129. Sono sul pianeta: 16767
Benzina rimanente: 10108. Sono sul pianeta: 9107
Benzina rimanente: 10097. Sono sul pianeta: 10559
Benzina rimanente: 10091. Sono sul pianeta: 3937
Benzina rimanente: 10073. Sono sul pianeta: 19527
Benzina rimanente: 10061. Sono sul pianeta: 3913
Benzina rimanente: 10053. Sono sul pianeta: 10865
Benzina rimanente: 10034. Sono sul pianeta: 5913
Benzina rimanente: 10032. Sono sul pianeta: 16834
Benzina rimanente: 10015. Sono sul pianeta: 18791
Benzina rimanente: 9978. Sono sul pianeta: 1036
Benzina rimanente: 9971. Sono sul pianeta: 3396
Benzina rimanente: 9944. Sono sul pianeta: 12909
Benzina rimanente: 9943. Sono sul pianeta: 16929
Benzina rimanente: 9926. Sono sul pianeta: 14586
Benzina rimanente: 9922. Sono sul pianeta: 6495
Benzina rimanente: 9899. Sono sul pianeta: 15794
Benzina rimanent

Benzina rimanente: 7831. Sono sul pianeta: 13785
Benzina rimanente: 7820. Sono sul pianeta: 10425
Benzina rimanente: 7803. Sono sul pianeta: 16829
Benzina rimanente: 7792. Sono sul pianeta: 8448
Benzina rimanente: 7773. Sono sul pianeta: 14175
Benzina rimanente: 7771. Sono sul pianeta: 8269
Benzina rimanente: 7764. Sono sul pianeta: 7442
Benzina rimanente: 7749. Sono sul pianeta: 15713
Benzina rimanente: 7725. Sono sul pianeta: 140
Benzina rimanente: 7715. Sono sul pianeta: 17427
Benzina rimanente: 7686. Sono sul pianeta: 14063
Benzina rimanente: 7674. Sono sul pianeta: 2086
Benzina rimanente: 7655. Sono sul pianeta: 3737
Benzina rimanente: 7646. Sono sul pianeta: 3107
Benzina rimanente: 7631. Sono sul pianeta: 17839
Benzina rimanente: 7613. Sono sul pianeta: 13881
Benzina rimanente: 7603. Sono sul pianeta: 6230
Benzina rimanente: 7596. Sono sul pianeta: 15478
Benzina rimanente: 7563. Sono sul pianeta: 13162
Benzina rimanente: 7545. Sono sul pianeta: 15418
Benzina rimanente: 7519. Sono

Benzina rimanente: 5621. Sono sul pianeta: 3823
Benzina rimanente: 5606. Sono sul pianeta: 7227
Benzina rimanente: 5579. Sono sul pianeta: 16792
Benzina rimanente: 5570. Sono sul pianeta: 1101
Benzina rimanente: 5567. Sono sul pianeta: 979
Benzina rimanente: 5549. Sono sul pianeta: 490
Benzina rimanente: 5537. Sono sul pianeta: 1121
Benzina rimanente: 5524. Sono sul pianeta: 11152
Benzina rimanente: 5507. Sono sul pianeta: 3112
Benzina rimanente: 5479. Sono sul pianeta: 11098
Benzina rimanente: 5468. Sono sul pianeta: 1802
Benzina rimanente: 5452. Sono sul pianeta: 12077
Benzina rimanente: 5450. Sono sul pianeta: 9575
Benzina rimanente: 5445. Sono sul pianeta: 4156
Benzina rimanente: 5405. Sono sul pianeta: 2728
Benzina rimanente: 5386. Sono sul pianeta: 1517
Benzina rimanente: 5372. Sono sul pianeta: 11768
Benzina rimanente: 5370. Sono sul pianeta: 1510
Benzina rimanente: 5351. Sono sul pianeta: 10899
Benzina rimanente: 5325. Sono sul pianeta: 6911
Benzina rimanente: 5312. Sono sul pi

Benzina rimanente: 3393. Sono sul pianeta: 5740
Benzina rimanente: 3385. Sono sul pianeta: 8251
Benzina rimanente: 3383. Sono sul pianeta: 17122
Benzina rimanente: 3382. Sono sul pianeta: 14422
Benzina rimanente: 3369. Sono sul pianeta: 14824
Benzina rimanente: 3357. Sono sul pianeta: 13182
Benzina rimanente: 3340. Sono sul pianeta: 17419
Benzina rimanente: 3339. Sono sul pianeta: 17740
Benzina rimanente: 3336. Sono sul pianeta: 5852
Benzina rimanente: 3330. Sono sul pianeta: 11330
Benzina rimanente: 3317. Sono sul pianeta: 14340
Benzina rimanente: 3307. Sono sul pianeta: 14662
Benzina rimanente: 3299. Sono sul pianeta: 15521
Benzina rimanente: 3282. Sono sul pianeta: 9767
Benzina rimanente: 3256. Sono sul pianeta: 11422
Benzina rimanente: 3245. Sono sul pianeta: 5294
Benzina rimanente: 3243. Sono sul pianeta: 13354
Benzina rimanente: 3227. Sono sul pianeta: 11889
Benzina rimanente: 3210. Sono sul pianeta: 2633
Benzina rimanente: 3200. Sono sul pianeta: 12911
Benzina rimanente: 3198. S

Benzina rimanente: 1035. Sono sul pianeta: 7062
Benzina rimanente: 1023. Sono sul pianeta: 6741
Benzina rimanente: 1012. Sono sul pianeta: 7865
Benzina rimanente: 1003. Sono sul pianeta: 14247
Benzina rimanente: 989. Sono sul pianeta: 17241
Benzina rimanente: 973. Sono sul pianeta: 4319
Benzina rimanente: 965. Sono sul pianeta: 18844
Benzina rimanente: 963. Sono sul pianeta: 13833
Benzina rimanente: 951. Sono sul pianeta: 4593
Benzina rimanente: 924. Sono sul pianeta: 994
Benzina rimanente: 916. Sono sul pianeta: 12451
Benzina rimanente: 899. Sono sul pianeta: 6256
Benzina rimanente: 892. Sono sul pianeta: 6277
Benzina rimanente: 889. Sono sul pianeta: 1962
Benzina rimanente: 877. Sono sul pianeta: 2780
Benzina rimanente: 864. Sono sul pianeta: 17746
Benzina rimanente: 839. Sono sul pianeta: 2294
Benzina rimanente: 827. Sono sul pianeta: 5223
Benzina rimanente: 818. Sono sul pianeta: 16292
Benzina rimanente: 817. Sono sul pianeta: 16222
Benzina rimanente: 815. Sono sul pianeta: 18569
B

In [25]:
for k in ContainersId:
    print("Valore finale container "+str(k)+": "+str(Containers[k].value) + " su "+ str(Containers[k].capacity))

Valore finale container 0: 5000 su 5000
Valore finale container 1: 2500 su 2500
Valore finale container 2: 10000 su 10000
Valore finale container 3: 5000 su 5000
Valore finale container 4: 3275 su 7000
